### * Getting Data from previous Model

In [1]:
import torch
from torch import nn

In [2]:
from initial_functions_from_01 import data_call,accuracy_fn,print_train_time,results_model_0

In [3]:
train_data,test_data,train_dataloader,test_dataloader=data_call()

In [4]:
len(train_data),len(test_data),len(train_dataloader),len(test_dataloader)

(60000, 10000, 1875, 313)

## 5. Setup Device agnostic-code(for using a GPU if there is one)

In [5]:
#Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 6. Model 1: Building a better model with non-linearity

In [6]:
#create a model with non-linear and linear layers
class FashionMNISTModelV1(nn.Module):
    def __init__(self, input_shape:int, hidden_units:int, output_shape:int):
        super().__init__()
        self.layer_stack=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_shape),
            nn.ReLU()
        )
    def forward(self, x: torch.Tensor):
        return self.layer_stack(x)

In [7]:
class_names=train_data.classes

In [8]:
torch.manual_seed(42)
model_1 = FashionMNISTModelV1(input_shape=784,
                             hidden_units=10,
                             output_shape=len(class_names)).to(device)
next(model_1.parameters()).device

device(type='cuda', index=0)

### 6.1 Setup loss,optimizer and evaluation Metrics 

In [9]:
loss_fn=nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(),lr=0.1)

In [10]:
def train_step(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              optimizer: torch.optim.Optimizer,
              accuracy_fn,
              device: torch.device=device):
    train_loss,train_acc=0,0
    model.to(device)
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y=X.to(device), y.to(device)
        
        # 1. forward pass
        y_pred = model(X)
        
        # 2 . Calucate loss
        loss=loss_fn(y_pred,y)
        train_loss+=loss
        train_acc+=accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))# go from logits to pred labels
        
        # 3 . optimizer zero grad
        optimizer.zero_grad()
        
        # 4 . Loss backwards
        loss.backward()
        
        # 5 . Optimizer step
        optimizer.step()
        
    #Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} || Train accuracy : {train_acc:.2f}%")
    
def test_step(data_loader: torch.utils.data.DataLoader,
             model: torch.nn.Module,
             loss_fn: torch.nn.Module,
             accuracy_fn,
             device: torch.device = device):
    test_loss, test_acc = 0,0
    model.to(device)
    model.eval()# Put model in eval mode
    # turn on inference context manager
    with torch.inference_mode():
        for X, y in data_loader:
            # send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1.forward pass
            test_pred = model(X)
            
            # 2.Calculate loss and accuracy
            test_loss+=loss_fn(test_pred,y)
            test_acc+=accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))
        #adjust the metrics
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [11]:
torch.manual_seed(42)
from tqdm.auto import tqdm
#Measure time
from timeit import default_timer as timer
train_time_start_on_gpu = timer()

epoch = 3
for epoch in tqdm(range(epoch)):
    print(f"Epoch: {epoch}\n-----")
    train_step(data_loader=train_dataloader,
              model=model_1,
              loss_fn=loss_fn,
              optimizer=optimizer,
              accuracy_fn=accuracy_fn)
    test_step(data_loader=test_dataloader,
             model=model_1,
             loss_fn=loss_fn,
             accuracy_fn=accuracy_fn)
train_time_end_on_gpu=timer()
total_train_time_model_1=print_train_time(start=train_time_start_on_gpu,end=train_time_end_on_gpu,
                                         device=device)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
-----
Train loss: 1.09199 || Train accuracy : 61.34%
Test loss: 0.95636 | Test accuracy: 65.00%

Epoch: 1
-----
Train loss: 0.78101 || Train accuracy : 71.93%
Test loss: 0.72227 | Test accuracy: 73.91%

Epoch: 2
-----
Train loss: 0.67027 || Train accuracy : 75.94%
Test loss: 0.68500 | Test accuracy: 75.02%

Train time on cuda: 30.476 seconds


In [12]:
torch.manual_seed(42)
def eval_model(model: torch.nn.Module,
              data_loader : torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    loss, acc = 0,0
    model.to(device)
    model.eval()# Put model in eval mode
    # turn on inference context manager
    with torch.inference_mode():
        for X, y in data_loader:
            # send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1.forward pass
            y_pred = model(X)
            
            # 2.Calculate loss and accuracy
            loss+=loss_fn(y_pred,y)
            acc+=accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
        #adjust the metrics
        loss /= len(data_loader)
        acc /= len(data_loader)
    return{"model_name": model.__class__.__name__,
          "model_loss": loss.item(),
          "model_acc": acc}

In [13]:
#Calculate model_1 result with device-agnostic code
model_1_result=eval_model(model=model_1, data_loader=test_dataloader, 
                         loss_fn=loss_fn, accuracy_fn=accuracy_fn, 
                         device=device)
model_1_result

{'model_name': 'FashionMNISTModelV1',
 'model_loss': 0.6850008368492126,
 'model_acc': 75.01996805111821}

In [14]:
model_0_results=results_model_0()
model_0_results

{'model_name': 'FashionMNISTV0',
 'model_loss': '0.4766390025615692',
 'model_acc': '83.42651757188499'}